In [1]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [2]:
!pip install transformers sentencepiece

     |████████████████████████████████| 2.5MB 7.7MB/s 
     |████████████████████████████████| 1.2MB 50.5MB/s 
     |████████████████████████████████| 3.3MB 49.0MB/s 
     |████████████████████████████████| 901kB 45.2MB/s 


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [4]:
test_path  = '/drive/MyDrive/ml/sequora/datasets/internal_data/sentiment/sentiment_test.csv'

In [5]:
test_data  = pd.read_csv(test_path)

In [6]:
labels = list(test_data['label'])

In [7]:
texts = list(test_data['text'])

In [8]:
!pip install transformers sentencepiece

In [9]:
class Classifier:

    def __init__(self, model_path, tokenizer_path):
        self.model = torch.load(model_path)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.max_len=256
        self.model.to(self.device)
    
    def predict(self, text):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }
        
        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)
        
        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0)
        )
        
        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

        return prediction

In [10]:
cls = Classifier(
    '/drive/MyDrive/ml/sequora/sentiment/model/sent_bert.pt',
    'cointegrated/rubert-tiny'
)

text = texts[0]

prediction = cls.predict(text)

print(prediction)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



2


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [11]:
pred_list = [cls.predict(text) for text in texts]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
from sklearn.metrics import precision_recall_fscore_support

In [13]:
precision_recall_fscore_support(labels, pred_list)

(array([0.87012987, 0.86764706, 0.83636364]),
 array([0.89333333, 0.83098592, 0.85185185]),
 array([0.88157895, 0.84892086, 0.8440367 ]),
 array([75, 71, 54]))